## eWindow Market Data - Deal Count and Avg. Price

This notebook uses the eWindow Market Data API to get Deal Counts and Price over time per Market/Product/Strip.


### Initial Setup


In [15]:
# uncomment below to install dependencies
# pip install spgci, pandas, plotly

In [3]:
import spgci as ci
from datetime import date, timedelta
import pandas as pd
from typing import Literal


In [13]:
# uncomment below to set your credentials
# ci.set_credentials("username", "password", "appkey")
ewmd = ci.EWindowMarketData()


### Getting the List of Markets

`get_markets` returns the list of markets you're entitled to.

Filtering down to markets with activity in the last 5 days.


In [5]:
markets = ewmd.get_markets()
filt = markets["max(order_date)"].dt.date >= date.today() - timedelta(days=5)
markets = markets[filt].reset_index(drop=True)
markets.sample(10)


,market,max(order_date)
33,USAC Distillates,2023-04-06
25,EU MTBE,2023-04-06
17,EU Ethanol,2023-04-06
35,USGC Gasoline,2023-04-06
3,ASIA FO (PVO),2023-04-06
6,Asia LNG Derivative,2023-04-05
12,ASIA MTBE,2023-04-06
31,US Midwest,2023-04-06
16,EU Brent CFD,2023-04-06
4,ASIA FO Swap,2023-04-06


### Getting Data

the `get_data` function will get `consummated` trades for a particular `market` and `date_lower` <= `order_time` <= `date_upper`.


In [6]:
def get_data(market: str, date_lower: date, date_upper: date):
    df = ewmd.get_botes(
        market=market,
        order_state=ewmd.OrderState.Consummated,
        order_time_gte=date_lower,
        order_time_lte=date_upper,
        paginate=True,
    )
    return df


df = get_data("EU BFOE", date(2023, 1, 1), date(2023, 3, 31))
df


,market,product,hub,strip,update_time,market_maker,order_type,order_state,buyer,seller,...,reference_order_id,order_platts_id,order_cancelled,order_derived,order_quantity_total,order_repeat,leg_prices,parent_deal_id,order_spread,order_state_detail
0,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Jun23,2023-03-31T15:29:31.730,Mercuria Energy Trading SA,Offer,consummated,Vitol SA,Mercuria Energy Trading SA,...,759115177.0,99402935,F,F,100000,None,None,None,F,consummated
1,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Jun23,2023-03-31T15:29:22.869,Mercuria Energy Trading SA,Offer,consummated,Vitol SA,Mercuria Energy Trading SA,...,759115177.0,99402928,F,F,100000,None,None,None,F,consummated
2,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Jun23,2023-03-31T15:29:10.699,Mercuria Energy Trading SA,Offer,consummated,Gunvor SA,Mercuria Energy Trading SA,...,103446557.0,99402899,F,F,100000,None,None,None,F,consummated
3,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Jun23,2023-03-30T15:29:58.868,Gunvor SA,Offer,consummated,Mercuria Energy Trading SA,Gunvor SA,...,978848165.0,99402618,F,F,100000,None,None,None,F,consummated
4,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Jun23,2023-03-30T15:29:52.905,Mercuria Energy Trading SA,Bid,consummated,Mercuria Energy Trading SA,Gunvor SA,...,521072649.0,99402604,F,F,100000,None,None,None,F,consummated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Mar23,2023-01-03T16:29:49.126,Mercuria Energy Trading SA,Offer,consummated,"Hartree Partners, LP",Mercuria Energy Trading SA,...,282371768.0,99402066,F,F,100000,None,None,None,F,consummated
311,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Mar23,2023-01-03T16:29:46.436,Vitol SA,Offer,consummated,Glencore Commodities Ltd.,Vitol SA,...,726710539.0,99402059,F,F,100000,None,None,None,F,consummated
312,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Mar23,2023-01-03T16:29:39.646,Mercuria Energy Trading SA,Offer,consummated,Chevron Products UK Limited,Mercuria Energy Trading SA,...,591131310.0,99401960,F,F,100000,None,None,None,F,consummated
313,[EU BFOE],Platts Cash BFOE,Cash Partials BFOE,Mar23,2023-01-03T16:29:36.960,Vitol SA,Offer,consummated,"Hartree Partners, LP",Vitol SA,...,591131310.0,99402053,F,F,100000,None,None,None,F,consummated


### Vizualize the Data

- Prepare the data for plotting - group by day; average of price, count of deals.
- Use Plotly to display in bar/line chart.


In [7]:
def group_df(df: pd.DataFrame) -> pd.DataFrame:
    grouped = (
        df.groupby(
            [pd.Grouper(key="order_time", freq="D", label="left"), "product", "strip"]
        )
        .agg(
            {
                "market": "count",
                "price": "mean",
                "deal_quantity": "mean",
                "price_unit": "max",
                "lot_unit": "max",
            }
        )
        .reset_index(drop=False)
    )

    grouped.columns = [
        "Day",
        "Product",
        "Strip",
        "Trade Count",
        "Avg. Price",
        "Trade QTY",
        "CCY",
        "UOM",
    ]
    return grouped


grouped = group_df(df)
grouped


,Day,Product,Strip,Trade Count,Avg. Price,Trade QTY,CCY,UOM
0,2023-01-03,Platts Cash BFOE,Mar23,7,83.217143,100.0,USD,bbl
1,2023-01-04,Platts Cash BFOE,Mar23,4,77.970000,100.0,USD,bbl
2,2023-01-05,Platts Cash BFOE,Mar23,4,79.142500,100.0,USD,bbl
3,2023-01-06,Platts Cash BFOE,Mar23,2,79.755000,100.0,USD,bbl
4,2023-01-09,Platts Cash BFOE,Apr23,1,79.970000,100.0,USD,bbl
...,...,...,...,...,...,...,...,...
79,2023-03-28,Platts Cash BFOE,Jun23,6,78.200000,100.0,USD,bbl
80,2023-03-28,Platts Cash BFOE,May23,2,78.800000,100.0,USD,bbl
81,2023-03-29,Platts Cash BFOE,Jun23,5,78.440000,100.0,USD,bbl
82,2023-03-30,Platts Cash BFOE,Jun23,10,78.361000,100.0,USD,bbl


In [8]:
import plotly.express as px
import plotly.graph_objects as go


In [12]:
def create_plot(
    df: pd.DataFrame,
    date_lower: date,
    date_upper: date,
    values: Literal["price", "count"] = "price",
) -> go.Figure:
    if values == "price":
        fig = px.line(
            df,
            x="Day",
            y="Avg. Price",
            color="Strip",
            facet_col="Product",
            facet_col_wrap=1,
            template="plotly_dark",
        )
    elif values == "count":
        fig = px.bar(
            df,
            x="Day",
            y="Trade Count",
            color="Strip",
            facet_col="Product",
            facet_col_wrap=1,
            template="plotly_dark",
        )

    fig.update_xaxes(
        matches="x",  # match x-axis across subplots, for easy comparison and brushing
        rangebreaks=[{"pattern": "day of week", "bounds": [6, 1]}],  # remove weekends
        range=[date_lower, date_upper],
    )

    fig.update_yaxes(matches=None)
    
    # make facet labels prettier
    fig.for_each_annotation(
        lambda a: a.update(text=a.text.split("=")[-1])
    )  

    fig.update_layout(
        height=240 * df["Product"].nunique(),
        title="eWindow Market Data - Trades by Date",
    )

    return fig


# notebook_connected is so that the chart renders in a non-interactive environment (such as nbviewer).
create_plot(grouped, date(2023, 1, 1), date(2023, 3, 31), values="count").show(
    "notebook_connected"
)


### Adding Interactivity

Using dash to handle selecting `market` and date range

**note:** this only works if you're running the notebook interactively (for example locally, or in Colab).


In [10]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output


In [11]:
app = JupyterDash(__name__)
app.layout = html.Div(
    [
        html.Div(
            [
                html.Div(
                    [
                        html.H2("Filters"),
                        html.Div([html.Hr()]),
                        html.Div(
                            [
                                dcc.RadioItems(
                                    options=[
                                        {"label": "Avg. Price", "value": "price"},
                                        {"label": "Deal Count", "value": "count"},
                                    ],
                                    value="price",
                                    inline=True,
                                    id="radio",
                                ),
                                dcc.Dropdown(
                                    id="markets-dropdown",
                                    clearable=False,
                                    value="EU BFOE",
                                    options=[
                                        {"label": m, "value": m}
                                        for m in markets["market"]
                                    ],
                                    style={
                                        "width": "288px",
                                    },
                                ),
                                dcc.DatePickerRange(
                                    id="date-picker",
                                    start_date=date.today() + timedelta(days=-30),
                                    end_date=date.today(),
                                ),
                            ],
                            style={
                                "marginTop": "20px",
                                "display": "flex",
                                "flex-direction": "column",
                                "gap": "25px",
                                "padding": "5px",
                            },
                        ),
                    ],
                ),
                html.Div(
                    [
                        dcc.Graph(id="output"),
                    ],
                    style={
                        "flex": 1,
                        "overflow": "scroll",
                    },
                ),
            ],
            style={"display": "flex", "gap": "30px", "height": "100vh"},
        ),
    ]
)


@app.callback(
    Output("output", "figure"),
    [
        Input("markets-dropdown", "value"),
        Input("radio", "value"),
        Input("date-picker", "start_date"),
        Input("date-picker", "end_date"),
    ],
)
def update(market: str, values, start_date: date, end_date: date):
    df = get_data(market, start_date, end_date)
    grouped = group_df(df)
    return create_plot(grouped, start_date, end_date, values)


app.run_server(mode="inline", use_reloader=False)


Dash is running on http://127.0.0.1:8050/



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.
